In [1]:
import pandas as pd

## 1. Data collection

In [2]:
Data=pd.read_csv("insurance_pre.csv")
Data

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [3]:
Data_set=pd.get_dummies(Data, drop_first=True)
Data_set

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [4]:
Data_set.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

## 2. Input and Output split

In [5]:
Independent=Data_set[['age', 'bmi', 'children', 'sex_male', 'smoker_yes']]
Independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [6]:
Dependent=Data_set[['charges']]
Dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


## 3.Split train and test data

In [7]:
#Here we are using Gird Search CV (cross validation) so no need to use train and test split, Gird will do that all the method.

## 4. Model creation

In [8]:
!pip install xgboost

In [15]:
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import KFold
from xgboost import XGBRegressor

p_grid={'n_estimators':[10,50,100], 'objective':['reg:squarederror','reg:absoluteerror']}

#In default CV take Kfold method to do cross validate we can mention the cv in code like below line
KFold=KFold(n_splits=5,shuffle=True, random_state=42)

grid=GridSearchCV(XGBRegressor(random_state=42), p_grid, cv=KFold, refit=True, verbose=3, n_jobs=-1)
grid.fit(Independent, Dependent)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, feature_weights=None,
                                    gamma=None, grow_policy=None,
                                    impo...
                                    learning_rate=None, max_bin=None,
                                    max_cat_threshold=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'n_estimators': [10, 50, 100],
                         'objective': ['reg:squarederror',
                                       'reg:absoluteerror']},
             verbose=3)

## 5. Evalution metric

In [16]:
#since we are using GridSearchCV so it will do Evalution the metric and give the best model

In [19]:
# Store the result in re variable
re=grid.cv_results_
print("The Score value for the best model {}".format(grid.best_params_),"Score :",grid.best_score_)

The Score value for the best model {'n_estimators': 10, 'objective': 'reg:squarederror'} Score : 0.8413903152636978


In [18]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_objective,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.027746,0.003939,0.005504,0.000721,10,reg:squarederror,"{'n_estimators': 10, 'objective': 'reg:squared...",0.873261,0.840238,0.876001,0.787672,0.829779,0.841390,0.032359,1
1,0.022714,0.001608,0.005643,0.000804,10,reg:absoluteerror,"{'n_estimators': 10, 'objective': 'reg:absolut...",0.826148,0.801333,0.728015,0.734709,0.781570,0.774355,0.037899,6
2,0.046220,0.001449,0.007198,0.000944,50,reg:squarederror,"{'n_estimators': 50, 'objective': 'reg:squared...",0.855597,0.807879,0.845985,0.765756,0.809774,0.816998,0.031910,4
3,0.063277,0.002848,0.006037,0.000002,50,reg:absoluteerror,"{'n_estimators': 50, 'objective': 'reg:absolut...",0.854958,0.828452,0.854302,0.773410,0.820245,0.826273,0.029813,3
4,0.071726,0.004769,0.008760,0.007530,100,reg:squarederror,"{'n_estimators': 100, 'objective': 'reg:square...",0.844114,0.792575,0.829525,0.753879,0.797102,0.803439,0.031462,5
5,0.081243,0.007595,0.009062,0.005075,100,reg:absoluteerror,"{'n_estimators': 100, 'objective': 'reg:absolu...",0.856270,0.825565,0.859515,0.773240,0.824399,0.827798,0.031015,2


In [25]:
age=int(input("Enter your age :"))
bmi=float(input("Enter your BMI : "))
children=int(input("Enter Children count:"))
Gen=int(input("Enter the gander if male enter 1, if female enter 0 :"))
smok=int(input("Enter if somker enter 1, if no somker enter 0 :"))

Enter your age :22
Enter your BMI : 27.900
Enter Children count:2
Enter the gander if male enter 1, if female enter 0 :0
Enter if somker enter 1, if no somker enter 0 :1


In [26]:
result=grid.predict([[age,bmi,children,Gen,smok]])

In [27]:
result

array([18948.797], dtype=float32)

In [28]:
print(float(result))

18948.796875
